In [37]:
import requests
import json
import pandas as pd
import folium

def tsunami_data():
    url = 'http://apis.data.go.kr/1741000/TsunamiShelter3/getTsunamiShelter1List'
    params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==', 'pageNo' : '1', 'numOfRows' : '1000', 'type' : 'json' }

    response = requests.get(url, params=params)
    json_ob = json.loads(response.content)
    body = json_ob['TsunamiShelter'][1]['row']
    body = pd.json_normalize(body)
    return body

In [53]:
sido_list = list(body['sido_name'].unique())
sido_list

['강원도', '경상북도', '부산광역시', '울산광역시', '전라남도']

In [45]:
def shelter_map(data):
        m = folium.Map(
        location=[data['lat'].mean(), data['lon'].mean()],
        zoom_start= 7
        )
        for idx in data.index:
                text = data.loc[idx, 'shel_nm'] + '<br>상세주소 :' + data.loc[idx, 'address'] + '<br>수용 가능 인원수 :' + str(data.loc[idx, 'shel_av'])  + '<br>해변으로부터 거리 :' + str(data.loc[idx, 'lenth']) + 'M' + '<br>해발 높이 :' + str(data.loc[idx, 'height']) + '<br>해변으로부터 거리 :' + str(data.loc[idx, 'lenth']) + '<br>내진적용여부 :' + data.loc[idx, 'seismic'] 
                folium.Marker([data.loc[idx, 'lat'], data.loc[idx, 'lon']], icon = folium.Icon(color="red"), tooltip = text).add_to(m)
        return m

In [54]:

def flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}{DocumentType}'
        response = requests.get(url)
        contents = response.text
        json_ob = json.loads(contents)
        body = json_ob['content']
        body = pd.json_normalize(body)
        return body
    

def flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType):
        url = f'https://api.hrfco.go.kr/{serviceKey}/{HydroType}/{DataType}/10M{DocumentType}'
        response = requests.get(url)
        contents = response.text
        # pp.pprint(response.content)
        json_ob = json.loads(contents)
        body = json_ob['content']
        # body = json_ob['response']['body']['item']
        body = pd.json_normalize(body)
        return body

def rainfall_api():
    serviceKey = 'A3A7BEB0-361E-4134-878C-BD8004204558'
    HydroType = 'rainfall'
    DataType = 'list'
    DocumentType = '.json'
    water_level_live = flowsiteAPI_livedata(serviceKey, HydroType, DataType, DocumentType)
    DataType = 'info'
    water_level = flowsiteAPI_data(serviceKey, HydroType, DataType, DocumentType)
    water_level['시도명'] = water_level['addr'].str.split(' ').str[0]
    water_level['lat'] = water_level['lat'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
    water_level['lon'] = water_level['lon'].apply(lambda x : int(x.split('-')[0]) + (int(x.split('-')[1]) / 60) + (int(x.split('-')[2]) / 3600) if len(x.split('-')) == 3 else x)
    rainfall_df = pd.merge(water_level, water_level_live, on = 'rfobscd', how = 'inner')
    return rainfall_df

In [59]:
import pandas as pd
df = pd.read_excel("C:/disaster_project/기상청41_단기예보 조회서비스_오픈API활용가이드_격자_위경도(20221027).xlsx")
df = df.groupby(['1단계','2단계'])[['격자 X', '격자 Y']].agg(pd.Series.mode).reset_index()
# df = df[df['1단계'] != '이어도'].reset_index(drop = True)
# df.columns = ['cd_nm', 'lon', 'lat']
# df.to_csv('korea_cd_latlon.csv', index = False, encoding = 'utf-8-sig')
df['격자 X'] = df['격자 X'].apply(lambda x : x[0] if str(type(x)) == "<class 'numpy.ndarray'>" else x)
df['격자 Y'] = df['격자 Y'].apply(lambda x : x[0] if str(type(x)) == "<class 'numpy.ndarray'>" else x)
df.to_csv('korea_weatherlocation_xy.csv', encoding = 'utf-8-sig', index = False)

In [60]:
df

,1단계,2단계,격자 X,격자 Y
0,강원도,강릉시,92,132
1,강원도,고성군,85,145
2,강원도,동해시,97,127
3,강원도,삼척시,98,125
4,강원도,속초시,87,141
...,...,...,...,...
245,충청북도,청주시상당구,69,106
246,충청북도,청주시서원구,69,106
247,충청북도,청주시청원구,69,107
248,충청북도,청주시흥덕구,68,106


In [ ]:
import requests

url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
params ={'serviceKey' : '서비스키', 'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'JSON', 'base_date' : '20210628', 'base_time' : '0630', 'nx' : '55', 'ny' : '127' }

response = requests.get(url, params=params)
print(response.content)

In [38]:
from datetime import datetime

base_date = datetime.now().strftime('%Y%m%d')
if int(datetime.now().strftime('%d')) > 30:
    base_time = datetime.now().strftime('%H30')
else :
    base_time = datetime.now().strftime('%H00')
    
base_date

'20221218'

In [52]:
type([1,2,3])

list

In [122]:
from datetime import datetime, timedelta
import requests
import json

# 시간 설정
base_datebf30 = datetime.now() - timedelta(minutes = 30)
base_date = base_datebf30.strftime('%Y%m%d')
if int(base_datebf30.strftime('%d')) > 30:
    base_time = base_datebf30.strftime('%H00')
else :
    base_time = base_datebf30.strftime('%H30')

print(base_date, base_time)
# api 호출준비(지역별 대기값)
korea_xy_df = pd.read_csv('C:/disaster_project/pages/using_data/korea_weatherlocation_xy.csv')

# 지역 선택
cd_nm_list = list(korea_xy_df['1단계'].unique())
cd_nm = '서울특별시'
sgg_nm_list = list(korea_xy_df[korea_xy_df['1단계'] == cd_nm]['2단계'].unique())
sgg_nm = '강서구'

# 격자 X, 격자 Y값 찾기
korea_xy_df = korea_xy_df[(korea_xy_df['1단계'] == cd_nm) & (korea_xy_df['2단계'] == sgg_nm)]
nx = korea_xy_df.iloc[0,2]
ny = korea_xy_df.iloc[0,3]

# api 호출
url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtFcst'
params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==',
        'pageNo' : '1', 'numOfRows' : '1000', 'dataType' : 'JSON', 'base_date' : base_date, 'base_time' : base_time, 'nx' : nx, 'ny' : ny }

# json csv 변환
response = requests.get(url, params=params)
contents = response.content
json_ob = json.loads(contents)
body = json_ob['response']['body']['items']['item']
body = pd.json_normalize(body)
body


20221219 0030


,baseDate,baseTime,category,fcstDate,fcstTime,fcstValue,nx,ny
0,20221219,0030,LGT,20221219,0100,0,58,126
1,20221219,0030,LGT,20221219,0200,0,58,126
2,20221219,0030,LGT,20221219,0300,0,58,126
3,20221219,0030,LGT,20221219,0400,0,58,126
4,20221219,0030,LGT,20221219,0500,0,58,126
5,20221219,0030,LGT,20221219,0600,0,58,126
6,20221219,0030,PTY,20221219,0100,0,58,126
7,20221219,0030,PTY,20221219,0200,0,58,126
8,20221219,0030,PTY,20221219,0300,0,58,126
9,20221219,0030,PTY,20221219,0400,0,58,126


In [95]:
base_date
base_time

'0000'

In [99]:
# 시간 설정
base_date = datetime.now().strftime('%Y%m%d')
if int(datetime.now().strftime('%d')) > 30:
    base_time = datetime.now().strftime('%H30')
else :
    base_time = datetime.now().strftime('%H00')

In [101]:
datetime.now().strftime('%Y%m%d%H')

'2022121900'

In [106]:
from datetime import datetime, timedelta
base_date = datetime.now() - timedelta(minutes = 30)
base_date

datetime.datetime(2022, 12, 18, 23, 47, 52, 419856)

In [40]:
import requests
import json
import pandas as pd
url='http://223.130.129.189:9191/getSgisDrinkWaterList/numOfRows=1000&pageNo=1&resultType=json'

response = requests.get(url)
json_ob = json.loads(response.content)

In [41]:
import requests
import json
import pandas as pd
url='http://223.130.129.189:9191/getSgisDrinkWaterList/numOfRows=60000&pageNo=1&resultType=json'

from pprint import pprint


response = requests.get(url)
json_ob = json.loads(response.content)
body = json_ob['getSgisDrinkWaterList']['item']
body = pd.json_normalize(body)

body


#body=body[['SPOT_NM','DO_NM','CTY_NM','ADRES']]

# for i in body.index:
#     if '구' in body.loc[i,'CTY_NM']
#     body.loc[i,'CTY_NM']=

#a=body[body['CTY_NM'].str.contains('시')]


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [1]:
import pandas as pd
df=pd.read_csv('C:\\Users\\user\\Desktop\\VS code\\disaster-main\\disaster-main\\pages\\using_data\\약수터.csv',encoding='utf-8-sig')
df=df.dropna()
df=df.reset_index(drop=True)

In [2]:
df

,SPOT_NM,DO_NM,CTY_NM,ADRES
0,잣절지하수공동시설,서울특별시,구로구,서울특별시 구로구 개봉동 2-61
1,잣절,서울특별시,구로구,서울특별시 구로구 개봉동 2-61
2,개봉3약수터,서울특별시,구로구,서울특별시 구로구 개봉동 53-3
3,오남,서울특별시,구로구,서울특별시 구로구 오류동 42-29
4,궁동,서울특별시,구로구,서울특별시 구로구 궁동 10-49
...,...,...,...,...
50950,장수,경기도,군포시,경기도 군포시 당동 48-1
50951,장수,경기도,군포시,경기도 군포시 당동 48-1
50952,오봉산,경기도,의왕시,경기도 의왕시 고천동 191-14
50953,오봉산,경기도,의왕시,경기도 의왕시 고천동 191-14


In [82]:
import pandas as pd
df=pd.read_csv('C:\\Users\\user\\Desktop\\VS code\\disaster-main\\disaster-main\\정수장코드.csv',encoding='utf-8-sig')
df=df.reset_index(drop=True)

from geopy.geocoders import Nominatim
geo_local = Nominatim(user_agent='South Korea')

# 위도, 경도 반환하는 함수
def geocoding(address):
    try:
        geo = geo_local.geocode(address)
        x_y = [geo.latitude, geo.longitude]
        return x_y

    except:
        return [0,0]

latitude = []
longitude =[]

for i in df['주소']:
    latitude.append(geocoding(i)[0])
    longitude.append(geocoding(i)[1])

df['위도']=latitude
df['경도']=longitude
print(df)
df.to_csv('정수장코드.csv',encoding='utf-8-sig')

      정수장     시도                             주소  정수장 코드         위도          경도
0   반월정수장    경기도           경기도 안산시 상록구 정재초교길 61     311  37.337753  126.866877
1   덕소정수장    경기도            경기도 남양주시 와부읍 안골로 78     317   0.000000    0.000000
2   고양정수장    경기도           경기도 고양시 일산동구 대주로 326     319  37.649395  126.813847
3   연초정수장   경상남도               경상남도 거제시 대금산로 95     331  34.971144  128.687105
4   구천정수장   경상남도            경상남도 거제시 거제중앙로 1333     332  34.847424  128.647563
5   사천정수장   경상남도           경상남도 사천시 축동면 수자원길 30     333   0.000000    0.000000
6   반송정수장   경상남도        경남 창원시 성산구 충혼로 224번길 18     335   0.000000    0.000000
7   온산정수장  울산광역시       울산광역시 울주군 온산읍 온산로 221-16     336  35.467550  129.328790
8   운문정수장   경상북도            경북 청도군 운문면 운문로 2490     340  35.657580  129.013500
9   고령정수장   경상북도            경북 고령군 다산면 다산로 1343     344  35.817403  128.439680
10  석성정수장   충청남도             충남 부여군 석성면 왕릉로 371     351  36.260841  126.990902
11  청주정수장   충청북도          충북 청주시 서원구 제2순환로 1571     

In [1]:
import pandas as pd
df=pd.read_csv('C:\\Users\\user\\Desktop\\VS code\\disaster-main\\disaster-main\\pages\\using_data\\약수터.csv',encoding='utf-8-sig')
df=df.dropna()
df=df.reset_index(drop=True)
df

,SPOT_NM,DO_NM,CTY_NM,ADRES
0,잣절지하수공동시설,서울특별시,구로구,서울특별시 구로구 개봉동 2-61
1,잣절,서울특별시,구로구,서울특별시 구로구 개봉동 2-61
2,개봉3약수터,서울특별시,구로구,서울특별시 구로구 개봉동 53-3
3,오남,서울특별시,구로구,서울특별시 구로구 오류동 42-29
4,궁동,서울특별시,구로구,서울특별시 구로구 궁동 10-49
...,...,...,...,...
50950,장수,경기도,군포시,경기도 군포시 당동 48-1
50951,장수,경기도,군포시,경기도 군포시 당동 48-1
50952,오봉산,경기도,의왕시,경기도 의왕시 고천동 191-14
50953,오봉산,경기도,의왕시,경기도 의왕시 고천동 191-14


In [2]:
df.drop_duplicates(['SPOT_NM','ADRES'],keep='first',inplace=True,ignore_index = True)

In [4]:
df.to_csv('위도.csv',encoding='utf-8-sig',index=False)

In [6]:
def tsunami_data():

    pageNo = 1

    numOfRows = 1000

    type = 'json'

    url = f'http://223.130.129.189:9191/getTsunamiShelter1List/numOfRows={numOfRows}&pageNo={pageNo}&type={type}'

    response = requests.get(url)

    json_ob = json.loads(response.content)

    body = json_ob['TsunamiShelter'][1]['row']

    body = pd.json_normalize(body)

    return body
tsunami_data()

NameError: name 'requests' is not defined

In [ ]:
with st.spinner('정보 조회 중입니다. 잠시 기다려주세요.'):
    try:
        # 공공데이터 조회
        url = 'http://apis.data.go.kr/B552657/ErmctInfoInqireService/getEmrrmRltmUsefulSckbdInfoInqire'
        params ={'serviceKey' : '3ouN4EKp4qGz+V76EbDHKehnbp5sYL0o19tpl5fAl2Q7s4ZosClGRfc1ENwk+2Px4QUPi4gCuCHGuG3kXFrs9w==', 'STAGE1' : cd_nm, 'STAGE2' : sgg_nm, 'pageNo' : '1', 'numOfRows' : '1000' }

        response = requests.get(url, params=params)
        content = response.text

        ### xml을 DataFrame으로 변환하기 ###
        #bs4 사용하여 item 태그 분리

        xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
        rows = xml_obj.findAll('item')

        # 각 행의 컬럼, 이름, 값을 가지는 리스트 만들기
        row_list = [] # 행값
        name_list = [] # 열이름값
        value_list = [] #데이터값

        # xml 안의 데이터 수집
        for i in range(0, len(rows)):
            columns = rows[i].find_all()
            #첫째 행 데이터 수집
            for j in range(0,len(columns)):
                if i ==0:
                    # 컬럼 이름 값 저장
                    name_list.append(columns[j].name)
                # 컬럼의 각 데이터 값 저장
                value_list.append(columns[j].text)
            # 각 행의 value값 전체 저장
            row_list.append(value_list)
            # 데이터 리스트 값 초기화
            value_list=[]

        #xml값 DataFrame으로 만들기
        corona_df = pd.DataFrame(row_list, columns=name_list)

        # 데이터 가공
        corona_df = corona_df[['hvidate','hvec','hvoc','hvgc','hvamyn','dutyName','dutyTel3']]
        corona_df.columns = ['정보 업데이트 일시','응급실 가용현황', '수술실 가용현황', '입원실 가용현황', '구급차 가용여부', '기관명', '연락처']
        corona_df['응급실 가용현황'] = corona_df['응급실 가용현황'].astype('int')
        sum = 0
        for i in corona_df['응급실 가용현황']:
            if i > 0:
                sum += i
        st.write('')
        st.write('')
        col1, col2, col3, col4 = st.columns(4)
        col1.metric("지역 내 응급의료기관 수", str(len(corona_df)) + '개')
        col2.metric('지역 내 응급실 가용병상', '총 ' + str(sum) + '개')
        st.write('')
        st.write('')
        st.write(corona_df.reset_index(drop = True))
        # st.success('Done!')
    except Exception as E:
        st.write("😓죄송합니다. 해당 지역에 의료시설이 없습니다.")
        print(E)
        

In [87]:
#실시간 유량
import requests
import json
import pandas as pd
from pprint import pprint
import bs4

url='http://223.130.129.189:9191/getWaterFlux/sujCode=318&stDt=2022-12-21&stTm=10&edDt=2022-12-21&edTm=11'
#url='http://223.130.129.189:9191/getWaterFlux/sujCode=316&stDt=2022-12-01&stTm=00&edDt=2022-12-01&edTm=01&numOfRows=500&pageNo=1'
response = requests.get(url)
content = response.text

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
corona_df = pd.DataFrame(row_list, columns=name_list)

corona_df
#corona_df.to_csv('정수장.csv',encoding='utf-8-sig')

,dataItemDesc,dataItemDiv,dataItemTagsn,dataVal,fcltyMngNo,fcltyNm,itemUnit,no,occrrncDt
0,파주(정) 원수 유입유량,M,44381,"4,337",4148012318,파주정수장,㎥/h,2,2022122110
1,고양 파주(정) 정수 유출 유량 적산차,D,19166,533,4148012318,파주정수장,㎥,1,2022122110


In [80]:
#유량 정수장 코드
import requests

url = 'http://apis.data.go.kr/B500001/rwis/waterFlux/fcltylist/codelist'
params ={'serviceKey' : 'z5tZeY7iv8p2Ib1ApuO7q//wKERNpofeMjmWiun9zefOVRmdNnZkj9DCIiZoMTf3fhj6CEToyLj94bjSGZ4q4A==', 'fcltyDivCode' : '2' }

response = requests.get(url, params=params)
content = response.text

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
corona_df = pd.DataFrame(row_list, columns=name_list)

print(corona_df)
corona_df.to_csv('정수장코드.csv',encoding='utf-8-sig')

   fcltyMngNm sujCode
0       반월정수장     311
1       와부정수장     312
2       성남정수장     313
3       시흥정수장     314
4       일산정수장     315
5       수지정수장     316
6       덕소정수장     317
7       파주정수장     318
8       고양정수장     319
9       연초정수장     331
10      구천정수장     332
11      사천정수장     333
12      반송정수장     335
13      온산정수장     336
14      자인정수장     339
15      운문정수장     340
16      고령정수장     344
17      석성정수장     351
18      청주정수장     353
19      아산정수장     354
20      천안정수장     355
21      송전정수장     356
22      공주정수장     357
23      보령정수장     359
24      부안정수장     360
25      산성정수장     363
26      화순정수장     364
27      군산정수장     365
28      대불정수장     366
29      고산정수장     367
30      별량정수장     368
31      황지정수장     371
32      금산정수장     372
33      학야정수장     378
34      충주정수장     380
35      밀양정수장     381
36      양산정수장     382
37      동화정수장     385
38      덕정정수장     386
39      평림정수장     387
40      구미정수장     866


In [72]:
#SPI 가뭄지수정보
import requests
import json
import pandas as pd
from pprint import pprint
import bs4

import datetime
dt_now = datetime.datetime.now()
# 날짜만 취득

# 2020-09-02
date=str(dt_now.date()).replace('-','')
date=int(date)-1
print(date)

url=f'http://223.130.129.189:9191/getAnalsInfoList/pageNo=1&numOfRows=100&hjdCd=1120052000&stDt={date}&edDt=20221220'
response = requests.get(url)
content = response.text

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

row_list = [] # 행값
name_list = [] # 열이름값
value_list = [] #데이터값

for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
corona_df = pd.DataFrame(row_list, columns=name_list)

print(corona_df)
#corona_df.to_csv('정수장.csv',encoding='utf-8-sig')


20221219
      anldt anlrst anlval    dv       hjdcd
0  20221219   보통습윤   1.15  SPI6  1120052000
1  20221219   보통습윤   1.01  SPI3  1120052000


In [ ]:
response = requests.get(url)
json_ob = json.loads(response.content)
body = json_ob['getSgisDrinkWaterList']['item']
body = pd.json_normalize(body)

body


#body=body[['SPOT_NM','DO_NM','CTY_NM','ADRES']]

# for i in body.index:
#     if '구' in body.loc[i,'CTY_NM']
#     body.loc[i,'CTY_NM']=

#a=body[body['CTY_NM'].str.contains('시')]


In [97]:
def flux(df):
    dt_now = datetime.datetime.now()
    hour=dt_now.hour
    date=str(dt_now.date()).replace('-','')
    date=int(date)-1
    row_list = [] # 행값
    value_list = [] #데이터값
    
    for i in df:
        url=f'http://223.130.129.189:9191/getWaterFlux/sujCode={i}&stDt={date}&stTm={hour}&edDt={date}&edTm={hour+1}'
        response = requests.get(url)
        content = response.text

        xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
        rows = xml_obj.findAll('item')

        name_list = [] # 열이름값
        
        for i in range(0, len(rows)):
            columns = rows[i].find_all()
            #첫째 행 데이터 수집
            for j in range(0,len(columns)):
                if i ==0:
                    # 컬럼 이름 값 저장
                    name_list.append(columns[j].name)
                # 컬럼의 각 데이터 값 저장
                value_list.append(columns[j].text)
            # 각 행의 value값 전체 저장
            row_list.append(value_list)
            # 데이터 리스트 값 초기화
            value_list=[]

        #xml값 DataFrame으로 만들기
        flux_df = pd.DataFrame(row_list, columns=name_list)
        
        #corona_df.to_csv('정수장.csv',encoding='utf-8-sig')
    return flux_df

a=[316,318]
flux(a)

,dataItemDesc,dataItemDiv,dataItemTagsn,dataVal,fcltyMngNo,fcltyNm,itemUnit,no,occrrncDt
0,수지(정) 송수 유량,F,4931,"23,431",4146512316,수지정수장,㎥/h,4,2022122012
1,수도권 수지(정) 5단계 송수유량 적산차,D,4933,230,4146512316,수지정수장,㎥,3,2022122012
2,수도권 수지(정) 5단계 송수유량 적산차,D,4933,232,4146512316,수지정수장,㎥,2,2022122011
3,수지(정) 송수 유량,F,4931,"22,780",4146512316,수지정수장,㎥/h,1,2022122011
4,고양 파주(정) 정수 유출 유량 적산차,D,19166,449,4148012318,파주정수장,㎥,4,2022122012
5,파주(정) 원수 유입유량,M,44381,"5,921",4148012318,파주정수장,㎥/h,3,2022122012
6,파주(정) 원수 유입유량,M,44381,"6,013",4148012318,파주정수장,㎥/h,2,2022122011
7,고양 파주(정) 정수 유출 유량 적산차,D,19166,543,4148012318,파주정수장,㎥,1,2022122011


In [98]:
flux_df = pd.DataFrame()
flux_df

""


,drghtFrcstPresnatnDt,frcstFarm,frcstFarmMsg,frcstLiv,frcstLivMsg,sigunNm
0,202212,정상,-\r,정상,-,가평군
1,202212,정상,-\r,정상,-,강릉시
2,202212,정상,-,정상,-,강진군
3,202212,정상,-,정상,-,강화군
4,202212,정상,-\r,정상,-,거제시
...,...,...,...,...,...,...
95,202212,정상,-,정상,-,영양군
96,202212,정상,-,정상,-,영월군
97,202212,정상,-\r,정상,-,영주시
98,202212,정상,-\r,정상,-,영천시


In [149]:
def drought():
    dt_now = datetime.datetime.now()
    hour=dt_now.hour
    date=str(dt_now.date()).replace('-','')
    date=int(date)-1

    url='http://223.130.129.189:9191/getInfoList/numOfRows=200&pageNo=1&_type=json&stDt=202212&edDt=202213'

    response = requests.get(url)
    json_ob = json.loads(response.content)
    body = json_ob['response']['body']['items']['item']
    df = pd.json_normalize(body)
    df=df[['sigunNm', 'frcstFarm', 'frcstFarmMsg', 'frcstLiv','frcstLivMsg']]
    df.rename(columns={'sigunNm':'시군구','frcstFarm':'생활 및 공업용수 가뭄 정보','frcstFarmMsg':'가뭄메시지','frcstLiv':'농업용수 가뭄정보','frcstLivMsg':'가뭄메시지'},inplace=True)
    df=df[df['농업용수 가뭄정보']!='정상']
    return df

drought()

TypeError: string indices must be integers

In [123]:

#url=f'http://223.130.129.189:9191/getWaterFlux/sujCode={i}&stDt={date}&stTm={hour}&edDt={date}&edTm={hour+1}'
url='http://223.130.129.189:9191/getInfoList/numOfRows=100&pageNo=1&_type=json&stDt=202212&edDt=202212'
response = requests.get(url)
content = response.text

xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows = xml_obj.findAll('item')

name_list = [] # 열이름값
row_list = [] # 행값
value_list = [] #데이터값
for i in range(0, len(rows)):
    columns = rows[i].find_all()
    #첫째 행 데이터 수집
    for j in range(0,len(columns)):
        if i ==0:
            # 컬럼 이름 값 저장
            name_list.append(columns[j].name)
        # 컬럼의 각 데이터 값 저장
        value_list.append(columns[j].text)
    # 각 행의 value값 전체 저장
    row_list.append(value_list)
    # 데이터 리스트 값 초기화
    value_list=[]

#xml값 DataFrame으로 만들기
flux_df = pd.DataFrame(row_list, columns=name_list)
    
flux_df


""


In [151]:
dt_now = datetime.datetime.now()
date=str(dt_now.date()).replace('-','')
str(date)[:-2]

'202212'

In [157]:
def drought():

    dt_now = datetime.datetime.now()
    date=str(dt_now.date()).replace('-','')
    date=str(date)[:-2]

    url=f'http://223.130.129.189:9191/getInfoList/numOfRows=200&pageNo=1&_type=json&stDt={date}&edDt={date}'

    response = requests.get(url)
    json_ob = json.loads(response.content)
    body = json_ob['response']['body']['items']['item']
    df = pd.json_normalize(body)
    df=df[['sigunNm', 'frcstFarm', 'frcstFarmMsg', 'frcstLiv','frcstLivMsg']]
    df.rename(columns={'sigunNm':'시군','frcstFarm':'생활 및 공업용수 가뭄 정보','frcstFarmMsg':'생활 및 공업용수 가뭄메시지','frcstLiv':'농업용수 가뭄정보','frcstLivMsg':'농업용수 가뭄메시지'},inplace=True)
    life_df=df[df['생활 및 공업용수 가뭄 정보']!='정상']
    list=life_df['시군'].to_list()
 
    return list

drought()

['나주시', '담양군', '무안군', '신안군', '장성군', '함평군']

In [ ]:
import requests

url = 'http://apis.data.go.kr/B500001/drghtIndexAnals/analsInfoList'
params ={'serviceKey' : '서비스키', 'pageNo' : '1', 'numOfRows' : '5', '_type' : 'xml', 'stDt' : '20211201', 'edDt' : '20211231' }

response = requests.get(url, params=params)
print(response.content)

In [171]:
df = pd.read_csv('C:/Users/user/Downloads/downloadContents (1).txt',encoding='cp949',sep = ',')

df

ParserError: Error tokenizing data. C error: Expected 14 fields in line 8, saw 16


In [191]:
user_1=[1,2,3,4,5]
del user_1[1]

In [198]:
with open('C:/Users/user/Downloads/downloadContents (1).txt',mode='r',encoding='cp949') as f:
    data = f.readlines()

column_name = data[0].strip().split(',')

data_split = [x.strip().split(',') for x in data[1:]]

#print(len(data_split))
for i in range(239,0,-1):
    if len(data_split[i])!=14:
        print(i)
        
ind=[225,189,178,163,162,159,157,152,143,141,129,97,96,92,83,17,14,7,6]
for i in ind:
    del data_split[i]

import pandas as pd

df = pd.DataFrame(data_split, columns = column_name)

df.to_csv('단수사례.csv',encoding='utf-8-sig',index=False)

225
189
178
163
162
159
157
152
143
141
129
97
96
92
83
17
14
7
6


In [164]:
import pandas as pd
df=pd.DataFrame(columns=["번호","지자체명","단수지역","단수사유","단수시작일시","단수시작시간","단수종료일시","단수종료시간","공사명","공사개요","작성부서","작성일","담당자","연락처"])
f=open('C:/Users/user/Downloads/downloadContents (1).txt',mode='r',encoding='cp949')
line=None
num=[]
while line!=' ' :
    line=f.readline()
    txt=line.split(',')
    print(txt)
    if not line: break

f.close()
df

['"번호"', '"지자체명"', '"단수지역"', '"단수사유"', '"단수시작일시"', '"단수시작시간"', '"단수종료일시"', '"단수종료시간"', '"공사명"', '"공사개요"', '"작성부서"', '"작성일"', '"담당자"', '"연락처"\n']
['"1"', '"단양군"', '"단양읍 후곡리"', '"상수관로 공사에 따른 단수"', '"2019-11-26"', '"09:40"', '"2019-11-26"', '"16:00"', '"후곡 하자보수"', '"단양군 하자보수로 인한 단수(후곡리)"', '"충주권지사"', '"2019-11-26"', '""', '""\n']
['"2"', '"단양군"', '"삼화동"', '"상수관로 누수"', '"2019-11-19"', '"14:00"', '"2019-11-19"', '"15:00"', '"누수복구"', '"배수관로 누수"', '"충주권지사"', '"2019-11-18"', '""', '""\n']
['"3"', '"단양군"', '"장발리"', '"상수관로 공사에 따른 단수"', '"2019-11-07"', '"00:00"', '"2019-11-07"', '"00:00"', '"상수도관로공사"', '"상수도관로공사"', '"충주권지사"', '"2019-11-07"', '""', '""\n']
['"4"', '"단양군"', '"어상천"', '"시설물 고장"', '"2019-09-02"', '"13:00"', '"2019-09-02"', '"18:00"', '"긴급단수"', '"긴급단수"', '"충주권지사"', '"2019-09-02"', '""', '""\n']
['"5"', '"단양군"', '"매포읍 안동길"', '"단양군 우덕교 교량확장공사 손괴사고"', '"2019-08-15"', '"12:30"', '"2019-08-15"', '"16:30"', '"단양군 우덕교 교량확장공사"', '"단양군 우덕교 교량확장공사"', '"충주권지사"', '"2019-08-15"', '"표명준"', '"043-420

,번호,지자체명,단수지역,단수사유,단수시작일시,단수시작시간,단수종료일시,단수종료시간,공사명,공사개요,작성부서,작성일,담당자,연락처
